### Check whether the data is faultyIf the file has the wrong number of time steps and has not been modified in the last hour, we directly remove it

In [1]:
# Python 3 unstable
import os
import xarray as xr
import numpy as np
import datetime

In [7]:
# Define all paths
inpath = '/work/ka1081/DYAMOND/ICON-2.5km'
outpath = '~/bd1179_work/DYAMOND/hvcg_data'

path_content_in = os.listdir(inpath)
path_content_out = os.listdir(outpath)

# files_out = [path_content_out[k] for k in range(len(path_content_out)) if 'clc' not in path_content_out[k] and '.nc' in path_content_out[k]]
files_out = [path_content_out[k] for k in range(len(path_content_out)) if '.nc' in path_content_out[k]]
files_in = [files_out[k].replace('.nc','.grb').replace('_R02B05', '') for k in range(len(files_out))]

*TEST 1*

In [8]:
def time_since_last_modified(fpath):
    '''
        In hours
    '''
    
    tfile = os.path.getmtime(fpath)
    temp_file = '~scratch/temp.txt'
    with open(temp_file, 'w') as file:
        file.write('')
    tnow = os.path.getmtime(temp_file)
    os.remove(temp_file)
    
    return (tnow - tfile)/3600

DYAMOND SUMMER

In [9]:
# All DYAMOND SUMMER files contain 8 timesteps of data.
# With the only exception of 20160911T000000Z.

# If the file has the wrong number of time steps and has not been modified in the last hour, we can directly remove it
remove_files = False

for k in range(len(files_out)):
    file = files_out[k]
    try:
        time_steps = len(xr.open_dataset(os.path.join(outpath, file)).time)
    except:
        if time_since_last_modified(os.path.join(outpath, file)) > 1:
            print('Could not open %s'%file)
        # If the file is faulty, remove it
        if time_since_last_modified(os.path.join(outpath, file)) > 1 and remove_files:
            os.remove(os.path.join(outpath, file))
            print('Removed %s'%file)
            continue
    
    # If the file contains the wrong number of time steps
    if (time_steps != 8 and '20160911T000000Z' not in file) or (time_steps != 1 and '20160911T000000Z' in file):
        print('%s has %d time steps'%(file, time_steps))
        
        # If the file additionally has not been modified in the last hour, we can directly remove it
        if time_since_last_modified(os.path.join(outpath, file)) > 1 and remove_files:
            os.remove(os.path.join(outpath, file))
            print('Removed %s'%file)

DYAMOND WINTER

In [10]:
# All DYAMOND WINTER files contain 7 timesteps of data.
# With the only exception of 20.1 and 23.1 (which consist of 8 timesteps each)

# If the file has the wrong number of time steps and has not been modified in the last hour, we can directly remove it
remove_files = False

for k in range(len(files_out)):
    file = files_out[k]
    try:
        time_steps = len(xr.open_dataset(os.path.join(outpath, file)).time)
    except:
        if time_since_last_modified(os.path.join(outpath, file)) > 1:
            print('Could not open %s'%file)
        # If the file is faulty, remove it
        if time_since_last_modified(os.path.join(outpath, file)) > 1 and remove_files:
            os.remove(os.path.join(outpath, file))
            print('Removed %s'%file)
            continue
    
    # If the file contains the wrong number of time steps
    if time_steps != 7 and '20200120000000' not in file and '20200123000000' not in file:
        print('%s has %d time steps'%(file, time_steps))
        
        # If the file additionally has not been modified in the last hour, we can directly remove it
        if time_since_last_modified(os.path.join(outpath, file)) > 1 and remove_files:
            os.remove(os.path.join(outpath, file))
            print('Removed %s'%file)

In [6]:
# Count all timesteps
paths = ['~/bd1179_work/DYAMOND/hcg_data/clw', '~/bd1179_work/DYAMOND/hcg_data/cli', '~/bd1179_work/DYAMOND/hcg_data/clc', '~/bd1179_work/DYAMOND/hcg_data/hus', '~/bd1179_work/DYAMOND/hcg_data/pa', '~/bd1179_work/DYAMOND/hcg_data/ta', '~/bd1179_work/DYAMOND/hcg_data/ua', '~/bd1179_work/DYAMOND/hcg_data/va']
for path in paths:
    path_content_out = os.listdir(path)
    files_out = [path_content_out[k] for k in range(len(path_content_out)) if '.nc' in path_content_out[k]]
    
    remove_files = False

    time_steps = 0
    for k in range(len(files_out)):
        file = files_out[k]
        time_steps += len(xr.open_dataset(os.path.join(path, file)).time)
    print(path)
    print(time_steps)

~/bd1179_work/DYAMOND/hcg_data/clw
619
~/bd1179_work/DYAMOND/hcg_data/cli
619
~/bd1179_work/DYAMOND/hcg_data/clc
619
~/bd1179_work/DYAMOND/hcg_data/hus
619
~/bd1179_work/DYAMOND/hcg_data/pa
619
~/bd1179_work/DYAMOND/hcg_data/ta
619
~/bd1179_work/DYAMOND/hcg_data/ua
619
~/bd1179_work/DYAMOND/hcg_data/va
619


In [4]:
file_count = {key: 0 for key in ['tot_qc_dia', 'tot_qi_dia', 'u_ml', '_v_ml', '_t_ml', 'qv_ml', 'pres_ml']}
files = os.listdir(outpath)

for file in files:
    for key in file_count.keys():
        if key in file:
            file_count[key] += 1
            break

print('Total number of files: %d'%len(files))

# There should be 42 per variable in the end!
file_count

Total number of files: 299


{'tot_qc_dia': 42,
 'tot_qi_dia': 42,
 'u_ml': 42,
 '_v_ml': 42,
 '_t_ml': 42,
 'qv_ml': 42,
 'pres_ml': 42}

*TEST 2*

In [87]:
# This is the test by comparing file size
# Actually it does suprisingly bad!!

# For the horizontally coarse-grained files we expect a **net reduction by a factor of around 260**
# For the copied (converted to nc) files we expect a **net reduction by a factor of around 0.027**

ratio_copied_files = {}
ratio_hcg_files = {}

for k in range(len(files_out)):
    file_in_size = os.path.getsize(os.path.join(inpath, files_in[k]))
    file_out_size = os.path.getsize(os.path.join(outpath, files_out[k]))
    
    if 'R02B05.nc' in files_out[k]:
        ratio_hcg_files[files_out[k]] = file_in_size/file_out_size
    else:
        ratio_copied_files[files_out[k]] = file_in_size/file_out_size